In [ ]:
import numpy as np
import pandas as pd
import os
import subprocess
import shutil
from pathlib import Path

def setup_freesurfer():
    freesurfer_home = '/home/localadmin/freesurfer'
    subjects_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia"
    os.environ['FREESURFER_HOME'] = freesurfer_home
    os.environ['SUBJECTS_DIR'] = subjects_dir
    
    setup_cmd = f"bash -c 'source {freesurfer_home}/SetUpFreeSurfer.sh; env'"
    try:
        process = subprocess.Popen(setup_cmd, stdout=subprocess.PIPE, shell=True)
        output, _ = process.communicate()
        for line in output.decode().split('\n'):
            if '=' in line:
                key, value = line.split('=', 1)
                os.environ[key] = value
        print(f"FreeSurfer environment initialized successfully")
        print(f"FREESURFER_HOME: {os.environ.get('FREESURFER_HOME')}")
        print(f"SUBJECTS_DIR: {os.environ.get('SUBJECTS_DIR')}")
    except Exception as e:
        print(f"Error setting up FreeSurfer: {e}")
        raise

# def determine_sign(param, contrast):
#     negative_params = ['ad', 'rd', 'md']
#     positive_params = ['ak', 'mk', 'rk', 'fa']
    
#     positive_contrasts = ['EP_metab_positive', 'HC_metab_negative', 'med_EPHC']
#     negative_contrasts = ['EP_metab_negative', 'HC_metab_positive', 'med_HCEP']
    
#     if param in negative_params:
#         return 'neg' if contrast in positive_contrasts else 'pos'
#     elif param in positive_params:
#         return 'pos' if contrast in positive_contrasts else 'neg'
    return None

def run_glmfit_sim(glmdir):
    folder_name = Path(glmdir).name
    param = folder_name.split('_')[2]
    contrast = folder_name.split('demeaned_')[-1].replace('.glmdir', '')
    
    print(f"Parameter: {param}")
    print(f"Contrast: {contrast}")
    
    # sign = determine_sign(param, contrast)
    # if not sign:
    #     print(f"Skipping {folder_name} - parameter not in target list")
    #     return False
        
    sig_path = os.path.join(glmdir, f'{contrast}', 'sig.mgh')
    print(f"Looking for sig.mgh at: {sig_path}")
    if not os.path.exists(sig_path):
        print(f"sig.mgh not found at {sig_path}")
        return False
    
    cmd = [
        'mri_glmfit-sim',
        '--glmdir', glmdir,
        '--perm', '1000', '2', 'abs',
        '--cwp', '0.05',
        # '--2spaces',
        '--bg', '3'
        
    ]
    print(' '.join(cmd))
    
    try:
        result = subprocess.run(
            ' '.join(cmd),
            shell=True,
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        print(f"Success for {glmdir}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error for {glmdir}:")
        print(e.stderr)
        return False

def main():
    setup_freesurfer()
    results_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_ins_results"
    
    glmdirs = [d for d in os.listdir(results_dir) if d.endswith('.glmdir')]
    for glmdir in glmdirs:
        full_path = os.path.join(results_dir, glmdir)
        if os.path.isdir(full_path):
            run_glmfit_sim(full_path)

if __name__ == "__main__":
    main()